In [2]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6,6)

from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

from keras.layers import Dense, Input, LSTM, Bidirectional, Activation, Conv1D, GRU, TimeDistributed
from keras.layers import Dropout, Embedding, GlobalMaxPooling1D, MaxPooling1D, Add, Flatten, SpatialDropout1D
from keras.layers import GlobalAveragePooling1D, BatchNormalization, concatenate
from keras.layers import Reshape, merge, Concatenate, Lambda, Average
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.initializers import Constant
from keras.layers.merge import add

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

Using TensorFlow backend.


In [0]:
df = pd.read_csv('/content/gdrive/My Drive/data_science/text_mining/train_values.csv', sep=',')

In [0]:
df_lab=pd.read_csv('/content/gdrive/My Drive/data_science/text_mining/train_labels.csv', sep=',')

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['doc_text'])
X = tokenizer.texts_to_sequences(df['doc_text'])
df['words'] = X

In [0]:
df['word_length'] = df.words.apply(lambda i: len(i))

In [9]:
df['word_length'] = df.words.apply(lambda i: len(i))
df = df[df.word_length >= 5]

df.head()

Index(['row_id', 'doc_text'], dtype='object')

In [0]:
df.words[0]

In [0]:
maxlen = 50
X = list(sequence.pad_sequences(df.words, maxlen=maxlen))

In [0]:
word_index = tokenizer.word_index

In [24]:
EMBEDDING_DIM = 100

embeddings_index = {}
f = open('/content/gdrive/My Drive/data_science/text_mining/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s unique tokens.' % len(word_index))
print('Total %s word vectors.' % len(embeddings_index))

Found 342859 unique tokens.
Total 400000 word vectors.


In [0]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index)+1,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=maxlen,
                            trainable=False)

In [0]:
Y=pd.read_csv('/content/gdrive/My Drive/data_science/text_mining/train_labels.csv',sep=',')

In [30]:
Y.columns

Index(['row_id', 'information_and_communication_technologies', 'governance',
       'urban_development', 'law_and_development', 'public_sector_development',
       'agriculture', 'communities_and_human_settlements',
       'health_and_nutrition_and_population', 'culture_and_development',
       'environment', 'social_protections_and_labor', 'industry',
       'macroeconomics_and_economic_growth',
       'international_economics_and_trade', 'conflict_and_development',
       'finance_and_financial_sector_development',
       'science_and_technology_development', 'rural_development',
       'poverty_reduction', 'private_sector_development', 'informatics',
       'energy', 'social_development', 'water_resources', 'education',
       'transport', 'water_supply_and_sanitation', 'gender',
       'infrastructure_economics_and_finance'],
      dtype='object')

In [0]:
Y=Y.drop('row_id',axis=1)

In [37]:
Y.shape[1]

29

In [0]:
X = np.array(X)


# and split to training set and validation set

seed = 29
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [55]:
inp = Input(shape=(maxlen,), dtype='int32')
embedding = embedding_layer(inp)
stacks = []
for kernel_size in [2, 3, 4]:
    conv = Conv1D(64, kernel_size, padding='same', activation='relu', strides=1)(embedding)
    pool = MaxPooling1D(pool_size=3)(conv)
    drop = Dropout(0.5)(pool)
    stacks.append(drop)

merged = Concatenate()(stacks)
flatten = Flatten()(merged)
drop = Dropout(0.5)(flatten)
outp = Dense((Y.shape[1]), activation='softprop')(drop)

TextCNN = Model(inputs=inp, outputs=outp)
TextCNN.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: ignored

In [41]:
textcnn_history = TextCNN.fit(x_train, 
                              y_train, 
                              batch_size=128, 
                              epochs=100, 
                              validation_data=(x_val, y_val))

Train on 15064 samples, validate on 3766 samples
Epoch 1/100
15064/15064 [==============================] - 1s 93us/step - loss: 8.4580 - acc: 0.2127 - val_loss: 9.0045 - val_acc: 0.2116
Epoch 2/100
15064/15064 [==============================] - 1s 82us/step - loss: 8.4355 - acc: 0.2116 - val_loss: 9.0123 - val_acc: 0.1827
Epoch 3/100
15064/15064 [==============================] - 1s 83us/step - loss: 8.4129 - acc: 0.2122 - val_loss: 9.0056 - val_acc: 0.2114
Epoch 4/100
15064/15064 [==============================] - 1s 83us/step - loss: 8.3907 - acc: 0.2161 - val_loss: 9.0087 - val_acc: 0.1848
Epoch 5/100
15064/15064 [==============================] - 1s 82us/step - loss: 8.3671 - acc: 0.2169 - val_loss: 9.0058 - val_acc: 0.1845
Epoch 6/100
15064/15064 [==============================] - 1s 81us/step - loss: 8.3496 - acc: 0.2207 - val_loss: 9.0074 - val_acc: 0.1814
Epoch 7/100
15064/15064 [==============================] - 1s 81us/step - loss: 8.3319 - acc: 0.2185 - val_loss: 9.0013 - v

In [0]:
predicted = TextCNN.predict(x_val)

In [0]:
save=pd.DataFrame(predicted)

In [0]:
save.to_csv('/content/gdrive/My Drive/data_science/text_mining/123.csv',sep=',')

In [51]:
np.where(y_val.iloc[0,:]==1)

(array([12, 13, 15, 19]),)

In [54]:
y_val.iloc[0,:]

information_and_communication_technologies    0
governance                                    0
urban_development                             0
law_and_development                           0
public_sector_development                     0
agriculture                                   0
communities_and_human_settlements             0
health_and_nutrition_and_population           0
culture_and_development                       0
environment                                   0
social_protections_and_labor                  0
industry                                      0
macroeconomics_and_economic_growth            1
international_economics_and_trade             1
conflict_and_development                      0
finance_and_financial_sector_development      1
science_and_technology_development            0
rural_development                             0
poverty_reduction                             0
private_sector_development                    1
informatics                             

In [0]:
collist=list(y_val.columns)

In [53]:
collist[12]

'macroeconomics_and_economic_growth'

In [0]:
!mkdir  picture

In [0]:
rm --help

Usage: rm [OPTION]... [FILE]...
Remove (unlink) the FILE(s).

  -f, --force           ignore nonexistent files and arguments, never prompt
  -i                    prompt before every removal
  -I                    prompt once before removing more than three files, or
                          when removing recursively; less intrusive than -i,
                          while still giving protection against most mistakes
      --interactive[=WHEN]  prompt according to WHEN: never, once (-I), or
                          always (-i); without WHEN, prompt always
      --one-file-system  when removing a hierarchy recursively, skip any
                          directory that is on a file system different from
                          that of the corresponding command line argument
      --no-preserve-root  do not treat '/' specially
      --preserve-root   do not remove '/' (default)
  -r, -R, --recursive   remove directories and their contents recursively
  -d, --dir             remove e

# 新段落

# 新段落